In [1]:
import cv2
import numpy as np

In [2]:
image='DATA/dog.jpg'
config='DATA/yolov3.cfg'
weights='DATA/yolov3.weights'
classes='DATA/yolov3.txt'

In [3]:
def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

In [4]:
def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [5]:
image = cv2.imread(image)
Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392

In [7]:
Width, Height

(768, 576)

In [8]:
classes = None

In [9]:
with open('DATA/yolov3.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

In [11]:
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

In [12]:
net = cv2.dnn.readNet(weights,config)

In [13]:
blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

In [14]:
net.setInput(blob)
outs = net.forward(get_output_layers(net))
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4


In [15]:
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])


In [16]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

In [17]:
for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

In [18]:
cv2.imshow("object detection", image)
cv2.waitKey()

-1

In [ ]:
    
cv2.imwrite("object-detection.jpg", image)
cv2.destroyAllWindows()

In [ ]:
import cv2
import dlib
import argparse
import time

In [ ]:
# handle command line arguments
image='DATA/NarendraModireuters.jpg'
weights='DATA/mmod_human_face_detector.dat'

In [ ]:
# initialize hog + svm based face detector
hog_face_detector = dlib.get_frontal_face_detector()

In [ ]:
# initialize cnn based face detector with the weights
cnn_face_detector = dlib.cnn_face_detection_model_v1(weights)

In [ ]:
cap=cv2.VideoCapture(0)
while True:
    _,image=cap.read()
    faces_hog = hog_face_detector(image, 1)
    for face in faces_hog:
        x = face.left()
        y = face.top()
        w = face.right() - x
        h = face.bottom() - y

    # draw box over face
    cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
    cv2.imshow("face detection with dlib", image)
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break


In [ ]:
# apply face detection (hog)
cv2.destroyAllWindows()

In [ ]:
cap=cv2.VideoCapture(0)
while True:
    _,image=cap.read()
    faces_cnn = cnn_face_detector(image, 1)
    for face in faces_cnn:
        x = face.left()
        y = face.top()
        w = face.right() - x
        h = face.bottom() - y
    img_height, img_width = image.shape[:2]
    cv2.putText(image, "CNN", (img_width-50,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,255,0), 2)
    # draw box over face
    cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
    cv2.imshow("face detection with dlib", image)
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()